In [39]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils import to_categorical, plot_model
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

import cv2

#MLP ON CIFAR

# network parameters
batch_size = 128
num_classes = 10
epochs = 25
data_augmentation = True


# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
image_size = x_train.shape[1]
# print('xtrain shape: ', x_train[0].shape)
# import matplotlib.pyplot as plt
# plt.imshow(x_train[0])
# plt.show()
# cv2.imshow("train", x_train[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()
x_train = x_train.reshape(-1, 3072)
x_test = x_test.reshape(-1, 3072)
x_train = x_train.astype('float32') / 255;
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_size = 3072
max_batches = len(x_train) / batch_size


# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# model is a 3-layer MLP with ReLU and dropout after each layer
model = Sequential()
model.add(Dense(512, input_dim=input_size))
model.add(Activation('relu'))
#model.add(Dropout(0.40))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.08))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.12))
model.add(Dense(num_classes))
# this is the output for one-hot vector
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='mlp-cifar.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer with default lr=0.01
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    # train the network no data augmentation
    x_train = np.reshape(x_train, [-1, input_size])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    # we need [width, height, channel] dim for data aug
    x_train = np.reshape(x_train, [-1, image_size, image_size, 3])
    datagen = ImageDataGenerator(
        data_format="channels_last",
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0.0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally
        height_shift_range=0.0,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    datagen.fit(x_train)
    for e in range(epochs):
        print("Epoch: %d/%d" % (e+1, epochs))
        batches = 0
        for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
            x_batch = np.reshape(x_batch, [-1, input_size])
            if e == 24:
                model.fit(x_batch, y_batch, verbose=1)
            else:
                model.fit(x_batch, y_batch, verbose=0)
            batches += 1
            if batches >= max_batches:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

# Score trained model.
x_test = np.reshape(x_test, [-1, input_size])
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


x_train shape: (50000, 3072)
50000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_141 (Dense)            (None, 512)               1573376   
_________________________________________________________________
activation_141 (Activation)  (None, 512)               0         
_________________________________________________________________
dense_142 (Dense)            (None, 512)               262656    
_________________________________________________________________
activation_142 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_79 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_143 (Dense)            (None, 512)               262656    
_________________________________________________________________
activati